In [20]:
# importing function from a different ipny 
from ipynb.fs.full.dataset import spam_test_train_set
from ipynb.fs.full.dataset import confusion
import pandas as pd
import collections
import numpy as np
import re
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from functools import reduce
import pickle
import spacy
from spacy.tokenizer import Tokenizer
nlp = spacy.load("en_core_web_sm")
tokenizer = Tokenizer(nlp.vocab)
pd.options.display.max_columns = 1000
#https://machinelearningmastery.com/deep-learning-bag-of-words-model-sentiment-analysis/

In [21]:
[train_email, test_email] = spam_test_train_set()
emails = pd.concat([train_email, test_email])

2018-12-25 08:56:26,674   ERROR:mailparser.utils:273:      parse_received More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+by|\s+with(?! cipher)|\s+id|\s+for|\s+via|;)) in from anderson.doramail.com 207.12.141.100 by coliform.funmail.co.uk InterMail vK.4.04.00.00 658-589-341 license 893694jz34ml9wal1qv276h1o63n3u76 with ESMTP id <20034613789721.SXCH195.coliform@anderson.doramail.com> for <kruscit@enron.com>; Sun, 5 Aug 2001 20:34:59 -0600 Received: from Matthew 206.81.96.2 by anderson.doramail.com iPlanet Messaging Server 5.2 HotFix 1.14 built Mar 18 2003 with ESMTPA id <0OMZ008USAR04O@anderson.doramail.com> for kruscit@enron.com; Sun, 5 Aug 2001 23:34:59 -0300 IDT
2018-12-25 08:56:26,978   ERROR:mailparser.utils:273:      parse_received More than one match found for (?:(?:^|\s)from\s+(?P<from>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+by|\s+with(?! cipher)|\s+id|\s+for|\s+via|;)) in from anderson.doramail.com 207

In [22]:
#s=train_email.Body.iloc[800]
#p = "<!DOCTYPE html public"
#result =  re.match(p,s)  
#if result:
#    print('YES')

In [23]:
train_email.subject.iloc[:20]

4                        CP&L tariff changes (ER01-1807)
5                            IM draft - Florida Projects
15              News About Your VentureWire Subscription
20                                            8/3/01 MPR
47     TRRC Chairman MICHAEL WILLIAMS and Land Commis...
54         FW: Day of Caring - 5719 Ettrick, Aug 8, 2001
56                              RE: Practice Sunday at 1
58              News About Your VentureWire Subscription
65                                                   RE:
67                                       Capacity Center
69     TRRC Chairman MICHAEL WILLIAMS and Land Commis...
71     Order re. transfer of operational control to MISO
72                            Billing Procedure Question
75                        Editorial on Emissions Trading
77                                              Schedule
79                                Practice Thurs. at 6PM
87            FW: Aug. 6 -- BNA, Inc. Daily Labor Report
95     PJM Interconnection LLC 

In [24]:
## New code 
def iter_spacy(strs, tokenizer):
    words = []
    for s in strs:
        #doc = nlp(s)
        doc = tokenizer(s)
        tokens = [str(token) for token in doc]
        counts = collections.Counter(tokens)
        yield counts

In [25]:
bows_spacy = pd.DataFrame(iter_spacy(emails.subject, tokenizer )).fillna(0).astype(int)

In [26]:
bows_spacy.head(5)

,,,,,,,,,,!,!!,!!.,!!nprr,!One,!r7,!ְ,"""","""Chris.Stokley@enron.com""","""Daily","""H0me","""Live","""Re:","""Strategy""","""biggest""","""little","""ȸڰ""ڷ^^",#19,#38;,$,$$$,$$濭ǳ$$ȸϸ,$1,$1.49,$15-$99..,$20,$249.99,$275,$60...abadad,$60...alimahon,$60...arpine,$60...ikramul,$65,$75,$80....abdelkader,$80....alme,$99,$avings,%,&,&amp;,'01,'em,'uincey,(,"(""EESI"")","(""EPMI"")",(&#44305)Ű,(&#44305)̾Ʈ-30,(((Ưõ)))ī,(()),(),()ִ,(0.7%),(0.8%)%,(01-406),(01-409),(01-411),(1,(1000,(17,(19,"(6,500)","(BHall,",(Backhaul),"(Cia1is),",(ClearStation),(ER01-1807),(ER01-2327),(Full,(Incl.,(M),"(NNG,",(NOT),(Norman),(Number,(OTC:,(OTCBB:,(RP00-336),(Swap,(V13.2),(Violation/Notification,(and,(ialis,(if,(no,(ûȯ)@,(Ңͧ,),*,****,**CANCELED**,*-*-*-*-gїZ2@-*-*-*-*,*CANCELED,*EMCA*,*̷,+,+0000,+0100,+0300,",",",,,,",",,99%1",",Wichtig!!!",-,"-,-,@Mn黴dwpendant",--,"--gardening,",-.the_best_porn_site_in_the_net.-_now_with_webcams!.-,-0300,-0400,-0500,-0600,-0800,-ı-,-６千万円以下年収の方に大歓迎されています---論より証拠です。,.,..!!,...,....,...aivas,...atav,...̷δ..,.ڽŰ,/,///,/18/02,0.4%ݸ,0.7%,00:02:02,00:04:01,01/07/02,01/08/2002,01/09/2002,01/17/02,01/17/2002,01/18/2002,02,04:55:44,05:52:23,06,06/21/02,"07,",07/24/01,08/04/01,08/06/01,08/15/01,"09,",09/13/2001),096276,0RDER,0RIGINAL,0f,0n,0ppurtunities,1,1%,1-08-02.xls,1-18-02.xls,1-18-2002,1-8-2002,1-9,1-9-02,1-δ(/ݸ),1.29$,1.49$,1/10,1/10/02,1/16,1/17,1/17/01:,1/17/02,1/17/02:,1/17/2002,1/18/02,1/21,1/22,1/22/02,1/8/02,1/8/02;,1/9,1/9/02,1/9/02;,10,100%,100%!,100%Ⱑմϴ.,1000,1000s,10;,10K,10г,11-30,11-30-01,11/30,11/30/01,11:00,11;,12-2-01,12/31,12:14:03,"13,",14,14;,14th,14th!,15,1500,15:38:48,15;,16,"16,",16:10:42,16;,17,"17,",17;,18,"18,",1854,18:08:07,18;,18TJ-pvO@FAuWʱΫ~ק,1994,19;,1:30,1ȸ,2,20%,"2000(湮,","2000,",2001,2001+02,2002,2002!,"2002,",2002.xls,2003,2005,"2005-Vales,Pumps,Seals,Pipings,Fittings,Flow",20;,20г,20оȿ!@,"21,",21th,22,"22,","22-23,",2234,22;,"23,",23;,24;,24ð,25,27,27867,27997,282481,29,2:00,2;,2nd,3.25,30,300,3000,30г,"31,",3127B,31st,350,36,36һȯ,386794,39C1,3;,3rd,4,4-20-05,4.....,40%,401K,401k,4102,47088,4749153,4775,494403,4x2,5-10times,5.6,500%,501,5719,5;,5th,5õ,5õ!~,5г,6,6/20,6/25/02,6/602,611Ͻǽ,637,64%,64%-off,661292,68%-off,6;,6PM,6th,7,705512.1,75%,7556363,7:00,7;,7mV9,8,"8,",8-16-01,8/16/01,8/17/01,8/3/01,80%,840514,8:00,8;,8th,9,"9,",9-14-2001,9/14/01,90/110,92854570,9648078),99.99!!,9900,9;,:,:P:ntermin,":Soma:,",<<Concur,=,>>,?,?!!??!!?,??,@,@q吶yVт̑㏞z,A,A&M,A.M.,ACCOMMODATIONS,ACOMODAES,"AD00BE,","AD0BBE,",AD1PEX,"AD1PEX,",AEP,AGA,AGENT_,"AL11AS,","AL1AS,",ALL,AM,"AMB11EN,","AMB1EEN,",AND,AOL,APB,"APPLE,",APPROVE,APPROVED,ASAP!,ASME,ASSISTANCE,AT,ATC,ATM,ATTN:,AVAILABLE,AWAIT,AWARD,About,Accepted:,Access,Accessories,Account,Acquire,Action,Actions,Activities,Activity,Adaytum,Adjust,Administration,Advantage:,Advisory,Affairs,Affordable,Affrs,After,Against,Agency,Agenda,Aggressive,Agreement,Ahead,Airlines,Airways',Akamai,Alert,Alerting.,Algorithmics:,Alice,All,All-Nude,Allowance,Alternate,Alumni,Alumni:,Amended,Amendment,American,Americans,Americas,Ameritrade,An,Analyst,Ancillary,Andersen,Andrews,Anime,Announcement,Announces,Answer,Antigen,Antitrust,Antonio,Any,Anytime,Application,Applications,Appointment,Approval,Approves,April,Are,Arrangements,Article,As,Ass,Assignment,Associate,Asuwaneh,Attendees,Audit,Aug,Aug.,Aug16,August,Authorization,AutoReply:,Autobytel.com,Automation,"Autopsy""",Ava,Average,Avila,BACK,BBC,BC,BCAP,BCG,BEAR,BEST,"BNA,",BNP,BUSINESS,BY,Baja,Balance,Baltimore,Bank,Bankruptcy,Bar,Barnard,BarronsArti,...,taxes,tdwaterhouse_2000,tech,teens,telephone,tell,telling,terminated,tests,textbooks!,than,thank,that,the,their,them,themselves..are,then,these,they're,things,third,this,thought,through,time,time),time-ftx,time.,times,titles,tits,tmtgnzvnmmx,to,todd,told,tonight,too,top,top-quality,tori.kuykendall@enron.com,touch,track,trad,trade,transaction,transactions.,transfer,treasure,treat,tree,trial,trip|e,"true,",trying,ts,ttm,ttqjlpr,turbine,twnq,u,uacrypvkrsd,ufi,ug,"ulcers,",unbe

### Test-train split

In [27]:
bows_spacy_train = bows_spacy[:1600]
bows_spacy_test = bows_spacy[1600:]
y_train= emails['isspam'][:1600]
y_test= emails['isspam'][1600:]

In [28]:
from sklearn.decomposition import PCA
pca = PCA(n_components=16)

In [29]:
topic_vectors_train = pca.fit_transform(bows_spacy_train)
topic_vectors_test = pca.transform(bows_spacy_test)

### PCA

In [30]:
C = [16,160,1600]
accuracy =[]
prec = []
true_pos =[]
false_pos = []
false_neg =[]
true_neg = []
accuracy_test =[]
prec_test = []
true_pos_test =[]
false_pos_test = []
false_neg_test =[]
true_neg_test = []
for c in C:
    pca = PCA(n_components=c)
    topic_vectors_train = pca.fit_transform(bows_spacy_train)
    topic_vectors_test = pca.transform(bows_spacy_test)
    logreg = LogisticRegression(C=1)

    # Fit the classifier to the training data
    logreg.fit(topic_vectors_train, y_train)

    # Predict the labels of the test set: y_pred
    y_pred =logreg.predict(topic_vectors_test)
    y_pred_train =logreg.predict(topic_vectors_train)
    true_pos.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[0])
    false_pos.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[1])
    false_neg.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[2])
    true_neg.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[3])
    accuracy.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[4])
    prec.append(np.array((confusion(y_pred_train, y_train.reshape((1, 1600)))))[5])
    ##################################################################################
    true_pos_test.append(np.array((confusion(y_pred, y_test.reshape((1, 400)))))[0])
    false_pos_test.append(np.array((confusion(y_pred, y_test.reshape((1, 400)))))[1])
    false_neg_test.append(np.array((confusion(y_pred, y_test.reshape((1, 400)))))[2])
    true_neg_test.append(np.array((confusion(y_pred, y_test.reshape((1, 400)))))[3])
    accuracy_test.append(np.array((confusion(y_pred, y_test.reshape((1, 400)))))[4])
    prec_test.append(np.array((confusion(y_pred, y_test.reshape((1, 400)))))[5])

/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/titli/anaconda3/lib/python3.6

In [31]:
metrics = pd.DataFrame(columns = ['true_pos','false_pos','false_neg','true_neg','Accuracy', 'Precision'],\
                      index = ['C= {}'.format(int(c)) for c in C])

In [32]:
metrics.Accuracy = accuracy
metrics.Precision = prec
metrics.true_pos = true_pos
metrics.true_neg = true_neg
metrics.false_neg = false_neg
metrics.false_pos = false_pos

In [33]:
metrics.head()

,true_pos,false_pos,false_neg,true_neg,Accuracy,Precision
C= 16,451.0,36.0,349.0,764.0,0.759375,0.56375
C= 160,611.0,43.0,189.0,757.0,0.855000,0.76375
C= 1600,758.0,11.0,42.0,789.0,0.966875,0.94750


In [34]:
true_pos = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[0]
false_pos = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[1]
false_neg = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[2]
true_neg = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[3]
accuracy = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[4]
prec = np.array((confusion(y_pred, y_test.reshape((1, 400)))))[5] 

/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until
/home/titli/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
/home/titli/anaconda3/lib/python3.6/site-packages/ipykern

In [35]:
metrics_test = pd.DataFrame(columns = ['true_pos','false_pos','false_neg','true_neg','Accuracy', 'Precision'],\
                           index = ['C= {}'.format(int(c)) for c in C])
metrics_test.Accuracy = accuracy_test
metrics_test.Precision = prec_test
metrics_test.true_pos = true_pos_test
metrics_test.true_neg = true_neg_test
metrics_test.false_neg = false_neg_test
metrics_test.false_pos = false_pos_test
metrics_test.head()

,true_pos,false_pos,false_neg,true_neg,Accuracy,Precision
C= 16,101.0,7.0,99.0,193.0,0.7350,0.505
C= 160,130.0,31.0,70.0,169.0,0.7475,0.650
C= 1600,146.0,32.0,54.0,168.0,0.7850,0.730


### Rerun and save model

In [36]:
pca = PCA(n_components=160)
topic_vectors_train = pca.fit_transform(bows_spacy_train)
logreg = LogisticRegression(C=0.1)
logreg.fit(topic_vectors_train, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [37]:
# save the model to disk
filename = 'PCA_model.sav'
pickle.dump(logreg, open(filename, 'wb'))